<center><h1>Creating a New UMD Dining Hall Nutrition Database and Classifying Vegetarian Dishes</h1><h4>Brooke Rice, Mitchell Smith</h4></center><hr>

The University of Maryland (UMD) dining halls are famous with students for their occasionally questionable array of sustenance. Trying to find nutrition information for any given meal is also notoriously difficult due to the horrendously arranged online campus dining database. That's why we're going to do a walkthough of creating a new, accesible dining hall nutrition database and show how this data can be used to train an is-vegetarian food classifier. This will be an end-to-end tutorial covering all steps from data acquisition to generalization error estimation. Our key learning objectives are thus as follows...
1. (web scraping) get necessary nutrition facts from the UMD dining halls website
2. (data tidying) prepare a read-only web database for easily querying nutrition information on the fly
3. (machine learning) prepare a "contains meat" classifier for seeing whether a given dish contains meat using only UMD dining hall data, predict generalization

For the sake of this project, we will be using the requests library, BeautifulSoup, and Pandas to acquire and process data. Machine learning will be performed using scikit-learn. Nutrition data will be sourced from https://nutrition.umd.edu/, and our generalization estimation set will come from INSERT-OTHER-NUTRITION-DB. We feel this project is relevant in that it is producing a more accessible and directly useful nutrition database while demonstrating the end to end data acquisition to machine learning analysis process.
    
# Scraping the UMD dining hall nutrition data
### Part 1: Collecting nutrition links
Before going straight into the scraping code, check out https://nutrition.umd.edu/. After a couple clicks, you might notice that the UMD dining hall web pages attempt to display a dynmaic meal page based off the query date. Since we want our database to function regardless of the current date, we can start our search from the base page for the South Campus Dining Hall (without providing a date). We can then automatically pick up on the current day's meal information by grabbing each meal's current weblink. 

After taking care of the dynamic date query aspect, we are free to start pulling information about the foods provided for each meal (breakfast, lunch, and dinner). Each individual food item link will lead us to the nutrition facts label that we want, but these label pages do not list whether a dish is vegetarian! Instead, whether or not a food item contains meat is listed on the corresponding meal page alongside the link to an item's nutrition label.

Because of this, we need to manually iterate through the food items table (as opposed to grabbing all relevant links with a single call to find_all()) and record the separate information about whether a dish is vegetarian. We can then able to cycle through each meal to copy all of the unique nutrition label links. Note that we say "unique" because some foods may be served across multiple meals. We'll use a set to avoid duplicate scraping.

In [1]:
# import standard html scraping libraries
from bs4 import BeautifulSoup
import requests

# declare base link (used in most of the site's hyperlinks)
baseSite = 'https://nutrition.umd.edu/'

# request dining hall nutrition webpage from minimized URL
r = requests.get(f'{baseSite}shortmenu.aspx?locationNum=16&naFlag=1')
htmlTree = BeautifulSoup(r.content)

# find links to each of breakfast, lunch, and dinner menu pages
mealLinks = [baseSite + str(link['href']) 
             for link in htmlTree.find_all('a', href=True)
             if 'mealName' in str(link)]

# zip links together with meal names for clarity
mealNames = ['breakfast', 'lunch', 'dinner']
mealPages = list(zip(mealNames, mealLinks))

# prepare output structure for storing food links
nutritionLinks = []

# note that there is overlap betw meals for some food items, so use a set to avoid duplication
prevWork = set()

# and record food table attributes for ease of access
tableAttributes = {"align":"center", "border":"1", "width":"70%", "cellspacing":"0", 
                   "cellpadding":"0", "bordercolor":"gray", "bgcolor":"#FFFFFF"}

# get nutrition information on a meal by meal query basis
for mealPage in mealPages:
    # print basic information about each meal page
    print(f'{mealPage[0]}: {mealPage[1]}')
    # query each meal page for food item elements
    r = requests.get(mealPage[1])
    htmlTree = BeautifulSoup(r.content)
    # vegetarian information is provided next to nutrition link in table
    table = htmlTree.find_all('table', attrs=tableAttributes)[0]
    tableRows = table.find_all('tr')
    # so loop through all table entries
    foodLinks = []
    for entry in tableRows:
        # and skip those entries which do not contain a food link
        if 'href' not in str(entry) or 'RecNumAndPort' not in str(entry):
            continue
        # then label foodlinks as vegetarian or not
        link = [str(link['href'])
                for link in entry.find_all('a', href=True)
                if 'RecNumAndPort' in str(link)][0]
        link = baseSite + link
        # do not reprocess links
        if link in prevWork:
            continue
        prevWork.add(link)
        if 'vegetarian' in str(entry):
            foodLinks.append([True, link])
        else:
            foodLinks.append([False, link])
    # store links in output dict
    nutritionLinks.extend(foodLinks)

breakfast: https://nutrition.umd.edu/longmenu.aspx?sName=&locationNum=16&locationName=&naFlag=1&WeeksMenus=This+Week%27s+Menus&dtdate=12%2f11%2f2021&mealName=Lunch
lunch: https://nutrition.umd.edu/longmenu.aspx?sName=&locationNum=16&locationName=&naFlag=1&WeeksMenus=This+Week%27s+Menus&dtdate=12%2f11%2f2021&mealName=Dinner


### Part 2: collecting nutrition information
To actually get our nutrition data, we need to query each individual food item's nutrition label page. We can start by declaing the nutrition facts we want to grab for our new database, and then we can scrape for each of the desired nutrition label elements on a per-food-item basis. 

The nutrition facts are stored in nested HTML tables using span tags, so with a bit of string parsing we can pull them directly from each food item's span tag set. We do have to grab the food name and serving size separately since they are stored in different tables. We should also keep in mind that there are a handful of food items with no nutrition data (mostly vegan dishes), so we should be careful not to query them for data which doesn't exist.

After rendering each individual nutrition fact, we can append it to the dataframe we declared earlier (loading all of the pre-defined target nutrition facts). Since this is a lengthy process and we have no gaurantee that the dining hall's websites will maintain the same format forever, we should go ahead and save our final dataframe to a pickle file for safe keeping. This will allow us to continue performing analysis and hosting nutrition data even after the dining hall webpages go offline. It's also worth noting that the nutrition data above is for a single serving of the corresponding food item (which might be measured in ounces or in numbers of the food item itself).

In [2]:
# import pandas so that we can create a nutrition dataframe
import pandas as pd

# declare nutrition dataframe and desired information
stats = ['Total Fat', 'Total Carbohydrate.', 'Saturated Fat', 'Dietary Fiber', 'Trans Fat',
        'Total Sugars', 'Cholesterol', 'Sodium', 'Protein', 'Calories', 'Carbohydrates', 
        'Vitamin C', 'Is Vegetarian', 'Serving Size', 'Food']
df = pd.DataFrame(columns=stats)

# integer for getting unique indices
entryNum = 0

# also create a structure for recording the handful of foods with no nutrition information
missingInfo = {}

# iterate through each meals nutrition links
for vegFlag,link in nutritionLinks:
    # query each food item's nutrition page
    r = requests.get(link)
    htmlTree = BeautifulSoup(r.content)
    # grab title of food item
    title = htmlTree.find('div', {"class":"labelrecipe"}).contents[0]
    # edge case: certain vegan foods do not have nutrition information!
    if (len(htmlTree.findAll('div', {"class":"labelnotavailable"})) != 0):
        missingInfo[title] = link
        continue
    # else, grab serving size 
    servingSize = htmlTree.findAll('div', {"class":"nutfactsservsize"})[1].contents[0]
    # then get misc. nutrition facts from webpage
    rawNutritionFacts = htmlTree.find_all('span', {"class":"nutfactstopnutrient"})
    # format nutrition information
    nutritionFactsList = [['Is Vegetarian', vegFlag], ['Food', str(title)], ['Serving Size', servingSize]]
    for fact in rawNutritionFacts:
        fact = list(filter(lambda str: len(str) != 0, fact.text.split('\xa0')))
        if (len(fact) == 2):
            nutritionFactsList.append(fact)
    nutritionFacts = dict(nutritionFactsList)
    # create new dataframe entry!
    df.loc[entryNum] = nutritionFacts
    # and increment entry num
    entryNum += 1
    
# save dataframe to pickle file!
df.to_pickle('umd_nutrition.pkl')
    
# return resultant dataframe
df

,Total Fat,Total Carbohydrate.,Saturated Fat,Dietary Fiber,Trans Fat,Total Sugars,Cholesterol,Sodium,Protein,Calories,Carbohydrates,Vitamin C,Is Vegetarian,Serving Size,Food
0,18.2g,0g,7.1g,0g,0.8gram,0g,63.4mg,467.7mg,15.1gram,222.8kcal,0gram,0mg,False,1 each,Beef Burger
1,23g,0g,9.8g,0g,0.8gram,0g,80.3mg,358.1mg,18.4gram,276.8kcal,0gram,0mg,False,1 each,Cheddar Burger
2,1.6g,3.2g,0g,0g,0gram,2.1g,31.8mg,265.3mg,7.4gram,53.1kcal,3.2gram,0mg,False,1 ea,Cranberry Chicken Sausage Breakfast
3,5.2g,25.1g,2.1g,2.7g,0gram,1.4g,0mg,144mg,2.8gram,162.3kcal,25.1gram,7.2mg,True,4 oz,Garlic Herb Roasted Yukon Gold Potatoes
4,4.4g,1.7g,0.4g,0.2g,0gram,0.1g,0.2mg,19mg,0.4gram,46.9kcal,1.7gram,1.4mg,False,1 each,Grilled Basil Pesto Chicken Breast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,0.4g,10.3g,0g,3.1g,0gram,3.8g,0mg,24.8mg,3.5gram,59.4kcal,10.3gram,5.9mg,True,3 oz,Peas and Caramelized Red Onions
317,0.1g,4.6g,0g,2.2g,0gram,2.1g,0mg,26.2mg,1.7gram,21.8kcal,4.6gram,40.5mg,True,3 oz,Steamed Cauliflower
318,7.6g,20.5g,3.6g,1.4g,0.1gram,0.7g,0mg,174.1mg,2.6gram,159.4kcal,20.5gram,6.7mg,True,4 oz,Fresh Vegan Basil Cheddar Mashed Potatoes
319,27.8g,78.4g,13.3g,6.9g,0gram,2.9g,0mg,1632.7mg,12.7gram,609.1kcal,78.4gram,10.4mg,True,1 each,Mushroom Carmelized Onion Quesadilla


# Data tidying: cleaning the nutrition data
After some poking around in our dataframe, we notice that there are a handful of nutrition labels which simply drop nutrients that were not part of the corresponding food item (ie: grilled hotdogs do not include an entry for dietary fiber). We likewise need to account for NaN values by replacing them with a 0. We can also convert our string measurements into floats to simplify data analysis. To that end, we should grab our measurement units and store them elsewhere for later reference.

We do NOT explicitly convert serving size because it is not provided with consistent units. By this we mean that most food items have their servings measured in ounces, but others are measured in terms of individual food objects. We also leave the food name and "Is Vegetarian" columns as-is since they are inherently non-numeric.

In [40]:
# import regex for string parsing
import re

# record units used for nutrition measurements
units = dict(zip(stats, ['grams', 'grams', 'grams', 'grams', 'grams', 'grams', 
                     'milligrams', 'milligrams', 'grams', 'kcal', 'grams', 'milligrams', 'ounces', 'N/A']))

# clean NaNs
toUpdate = {}
# NaNs indicate that the label did not include a nutrient (b/c the food has none, ie. hotdogs and fiber)
for idx,row in df.iterrows():
    numNans = len([entry for entry in row if str(entry).lower() == 'nan'])
    newRow = [entry if str(entry).lower() != 'nan' else "0" for entry in row]
    if numNans != 0:
        toUpdate[idx] = newRow

# so replace NaNs with 0!
for idx,row in toUpdate.items():
    df.loc[idx] = row
        
# define function to convert string measurement to numeric
def convertToNumeric(string):
    try:
        # when re-running code, do not erase pre-converted values!
        return float(re.sub(r'[a-zA-Z]', '', string)) if type(string) == type("") else string
    except:
        # some labels drop missing nutrients
        return float(0)

# then update columns to reflect their numeric counterparts
for colName in df.columns:
    # do not convert inherently non-numeric columns!
    if (colName == "Food" or colName == "Serving Size" or colName == "Is Vegetarian"):
        continue
    df[colName] = [convertToNumeric(measurement) for measurement in df[colName]]

# return updated dataframe
df

,Total Fat,Total Carbohydrate.,Saturated Fat,Dietary Fiber,Trans Fat,Total Sugars,Cholesterol,Sodium,Protein,Calories,Carbohydrates,Vitamin C,Is Vegetarian,Serving Size,Food
0,18.2,0.0,7.1,0.0,0.8,0.0,63.4,467.7,15.1,222.8,0.0,0.0,False,1 each,Beef Burger
1,23.0,0.0,9.8,0.0,0.8,0.0,80.3,358.1,18.4,276.8,0.0,0.0,False,1 each,Cheddar Burger
2,1.6,3.2,0.0,0.0,0.0,2.1,31.8,265.3,7.4,53.1,3.2,0.0,False,1 ea,Cranberry Chicken Sausage Breakfast
3,5.2,25.1,2.1,2.7,0.0,1.4,0.0,144.0,2.8,162.3,25.1,7.2,True,4 oz,Garlic Herb Roasted Yukon Gold Potatoes
4,4.4,1.7,0.4,0.2,0.0,0.1,0.2,19.0,0.4,46.9,1.7,1.4,False,1 each,Grilled Basil Pesto Chicken Breast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,0.4,10.3,0.0,3.1,0.0,3.8,0.0,24.8,3.5,59.4,10.3,5.9,True,3 oz,Peas and Caramelized Red Onions
317,0.1,4.6,0.0,2.2,0.0,2.1,0.0,26.2,1.7,21.8,4.6,40.5,True,3 oz,Steamed Cauliflower
318,7.6,20.5,3.6,1.4,0.1,0.7,0.0,174.1,2.6,159.4,20.5,6.7,True,4 oz,Fresh Vegan Basil Cheddar Mashed Potatoes
319,27.8,78.4,13.3,6.9,0.0,2.9,0.0,1632.7,12.7,609.1,78.4,10.4,True,1 each,Mushroom Carmelized Onion Quesadilla


Note that we have chosen to use a Pandas dataframe for data storage. We reccommend using Pandas for these tasks because:
1. Pandas makes tabular data representation incredibly easy (fast queries, updates, column additions, reindexing, filtering, basic statistics...)
2. Pandas has incredible support across the industry (as the de facto standard for tabular data, can automatically interface with HTML sources, output to SQL, interface with numpy and scipy, scikit-learn and statsmodels integrations, etc)

# Creating a read-only web database
At this point in the process, we have already acquired and cleaned the desired nutrition data. With the backing dataset more or less complete, creating a database becomes trivial. Since we are not UMD dining hall workers, we shouldn't be able to create, update, or delete menu items. That said, to create a read-only database from our dataframe, we would typically recommend [bamboolib](https://bamboolib.8080labs.com/). It's the perfect tool for this sort of thing, yet we will be creating our own interface with [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) for the sake of easy TA access. 

Libraries aside, we want our new nutrition database to function just like UMD's- we should be able to query for whatever food we desire (by name) and get a quick nutrition facts sheet with allergens listed. To that end, we will need to define a simple string search for finding close food matches (ie. "fwench toast" should find "French Toast") as well as a label generation method (for outputting nutrition information). We then want our interface to make use of these functions for a searchable nutrition dataset.

In [64]:
# import widgets library
import ipywidgets as widgets
from IPython.display import display, clear_output
# import difflib for finding close string matches
import difflib

### QUERY LOGIC ###
# get food names as list for diffing
foods = [food.lower() for food in df["Food"]]

# create a simple search function (returns indices of similarly named foods)
def searchDataframe(foodName):
    indices = []
    # try getting close matches with difflib
    possibleMatches = difflib.get_close_matches(foodName, foods)
    # if unsuccessful, manually search for food name in food set
    if len(possibleMatches) == 0:
        # look for decreasingly small substrings of input name in dataset
        for i in range(0,len(foodName)):
            currSubString = foodName[i:len(foodName)].lower()
            possibleMatches = [food for food in foods
                               if currSubString in food]
            # break when a match is found
            if len(possibleMatches) != 0:
                break
            # try it in reverse as well
            currSubString = foodName[0:len(foodName) - i].lower()
            possibleMatches = [food for food in foods
                               if currSubString in food]
            # break when a match is found
            if len(possibleMatches) != 0:
                break
    # then return the indices of possible matches (or an empty list if no match found)
    return [df.index[df.Food.str.lower() == m] for m in possibleMatches]
    
### LABEL GENERATION / PRETTY PRINT ###
# pretty print method for creating nutrition labels from a dataframe index
def getLabel(index):
    # get the corresponding row
    row = df.iloc[index]
    # change index to the food name, and drop the food name entry
    row.index = row["Food"]
    row = row.drop(columns=["Food"])
    # then format each nutrition measurement with proper units
    label = pd.DataFrame()
    for nutrient in row.columns:
        # do not reformat serving size string!
        if nutrient == "Serving Size":
            label[nutrient] = row[nutrient][0]
        # do convert vegetarian flag -> yes or no
        elif nutrient == "Is Vegetarian":
            label[nutrient] = "Yes" if row[nutrient][0] else "No"
        # and do add units to numeric data
        else:
            label[nutrient] = pd.Series(f"{str(row[nutrient][0])} {str(units[nutrient])}")
    # set the food name and return!
    label.index = row.index
    return label
    
### USER INTERFACE ###
# create a search button
searchButton = widgets.Button(description="search")

# create a simple textbox for querying for food items
strIn = widgets.Text(
    placeholder='search for foods',
    disabled=False
)

# group search button + text input
searchWidg = widgets.Box([strIn, searchButton])
display(searchWidg)

# create a text output region for returning query results
strOut = widgets.Text(
    placeholder='(query results)',
    disabled=False
)
    
# define click event for searching
output = widgets.Output()
@output.capture()
def on_button_clicked(init):
    # clear previous output, find possible food matches
    clear_output()
    possibleMatchIndices = searchDataframe(strIn.value)
    # pretty print nutrition labels for each
    for index in possibleMatchIndices:
        display(getLabel(index).T)
    
# link search button to click event
searchButton.on_click(on_button_clicked)
display(output)

Box(children=(Text(value='', placeholder='search for foods'), Button(description='search', style=ButtonStyle()…

Output()

We would argue that the above search box is far more convenient than looking for any given food item's nutrition information on https://nutrition.umd.edu/. That said, we realize that a search box alone is not a full-fledged database. For the sake of this tutorial we will briefly demonstrate the creation of a proper SQL database using the previously recorded nutrition information.
## Aside: Creating a true CRUD database
A true database should implement Create, Read, Update, and Delete behavior (the CRUD standard). Since we chose to represent our data using a dataframe, we can make use of Pandas "to_sql()" method in order to create a SQL database from our nutrition records. We can then do a couple of demo queries to show off the new, SQL-based nutrition database.